In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing import text
from keras.preprocessing.sequence import pad_sequences
from keras import layers
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import string
import re
from keras.models import Sequential
import tensorflow as tf
filename="2l_labeled.csv"

In [ ]:

f=open(filename,"r",encoding="utf8")
ids=[]
x=[]
cats=[]
for line in f:
    fileid,review,cat=line.split(',')
    ids.append(fileid)
    x.append(review)
    if cat=="pos\n":
        cats.append(1)
    elif cat=="neg\n":
        cats.append(0)
y=cats
f.close()

In [ ]:
def clean_str(string):
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()
X=[]
for i in x:
    X.append(clean_str(i))

tokenizer = text.Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ')
tokenizer.fit_on_texts(X)
xx=[]
for review in X:
    translator=review.maketrans("","",string.punctuation)
    xx.append(review.translate(translator))

x_train, x_test, y_train, y_test = train_test_split(xx, y, test_size=0.1)
xtrain= tokenizer.texts_to_sequences(x_train)
xtest= tokenizer.texts_to_sequences(x_test)
word_index=tokenizer.word_index

maxlen=1000
xtrain=pad_sequences(xtrain,padding='pre', maxlen=maxlen)
xtest=pad_sequences(xtest,padding='pre', maxlen=maxlen)

In [ ]:
filename="glove.txt"
embeddings_index = {}
f=open(filename,"r",encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
EMBEDDING_DIM=100
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
# CNN + LSTM


model=Sequential()
model.add(layers.Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=maxlen,
                            trainable=True))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(64, 5, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(128, 5, activation='relu'))

model.add(layers.LSTM(units=50,return_sequences=True))
model.add(layers.LSTM(units=10))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(8))
model.add(layers.Dense(1, activation="sigmoid"))
model.compile(optimizer="adam", loss="binary_crossentropy",
     metrics=['accuracy'])
model.fit(xtrain,y_train, epochs=10, batch_size=128, verbose=2)

loss,acc=model.evaluate(xtest,y_test)
print("Test accuracy:",acc)
l=(model.predict(xtest))


In [ ]:
# GUI with a blank slot to enter review and get classification


def clicked():
    global text
    X=text.get()
    text2=[]
    text3=[]
    text4=[]

    translator=X.maketrans("","",string.punctuation)
    text1=(X.translate(translator))

    result= re.sub(r'\d+', '', text1)
    text2.append(result)

    tokenizer = Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ')
    tokenizer.fit_on_texts(text2)
    maxlen=1000
    text3=(tokenizer.texts_to_sequences(text2))
    text4=pad_sequences(text3,padding='pre', maxlen=maxlen)
    p=model.predict(text4)
    print(p)
    if np.round(p[0])==0:
        label_2.configure(text="neg")
    elif np.round(p[0])==1:
        label_2.configure(text="pos")
    
   
       
from tkinter.ttk import *
from tkinter import *
root=Tk()
root.title("Sentiment Analysis")
root.geometry('500x250')
topframe=Frame(root)
topframe.pack()
label_1 = Label(topframe,text="review_text")
label_1.pack(side=LEFT)
text = Entry(topframe)
text.pack(fill=X)
text.focus_set()
bottomframe=Frame(root)
bottomframe.pack(side=BOTTOM)
button = Button(bottomframe,text="Submit", command=clicked)
button.pack()

label_2 = Label(bottomframe,text="category")
label_2.pack(side=LEFT)


root.mainloop()
